<a href="https://colab.research.google.com/github/mdjamina/m1_ml_lang_detector/blob/main/ML_lang_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus
[Tatoeba](https://tatoeba.org/fr/downloads
) est une collection de phrases et de traductions.

Extraction de l'archive téléchargée 

In [17]:
!unzip ./data/data.zip

Archive:  ./data/data.zip
warning [./data/data.zip]:  zipfile claims to be last disk of a multi-part archive;
  attempting to process anyway, assuming all parts have been concatenated
  together in order.  Expect "errors" and warnings...true multi-part support
  doesn't exist yet (coming soon).
file #1:  bad zipfile offset (local header sig):  4


## Chargement du corpus

In [1]:
import pandas as pd


data = pd.read_pickle("./data/data.pkl") 


data

,content,lang
0,我們試試看！,zho
1,我该去睡觉了。,zho
2,你在干什麼啊？,zho
3,這是什麼啊？,zho
4,今天是６月１８号，也是Muiriel的生日！,zho
...,...,...
10262652,Utálom a melóm!,hun
10262653,Ich habe den Ersatzschlüssel verloren.,deu
10262654,Mibe' fogadjunk?,hun
10262655,Utálok itt lenni!,hun


# pre-processing

## TODO

In [2]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data['content'], data['lang'], test_size=0.33, train_size=0.1, random_state=1) 
# ramdom_state = meme corpus (meme decoupage, decouper en 2 train et test le corpus, car on a besoin de faire les tests)

In [13]:
y_train.value_counts()

eng    157100
rus     90726
ita     80533
tur     72168
ber     58763
deu     58278
kab     51139
fra     49837
por     39536
spa     36408
hun     35480
jpn     21911
heb     19698
ukr     17565
nld     15472
fin     13775
pol     11538
zho      8451
mkd      7728
ces      6259
dan      5312
hbs      5188
swe      4552
ara      4170
lat      4159
ell      3362
ron      2723
fas      2511
bul      2463
vie      2137
slk      1607
nor      1530
hin      1429
kor       906
tha       484
est       389
lav       287
gle       273
hye       216
swa        52
Name: lang, dtype: int64

# Model

In [3]:
import re
def ngram_2c(text):
  #text = text.replace(' ','-')
  return [re.sub('[ ]?','',i+j) for (i,j) in zip(*[text[i:] for i in range(2)])]
  #ng = [re.sub('[ ]?','',i+j+k) for (i,j,k) in zip(*[text[i:] for i in range(3)])]
  #ng = [i+j+k+m for (i,j,k,m) in zip(*[text[i:] for i in range(4)]) if j!=' ' or k!=' ']
  #for (l,m) in zip(*[text[i:] for i in range(2)]):
   # gram=remove_punctuation(l+m)


import re
def to_char(text):
  #text = text.replace(' ','-')
  return [c for c in text if c not in ' \t\n']
  #ng = [re.sub('[ ]?','',i+j+k) for (i,j,k) in zip(*[text[i:] for i in range(3)])]
  #ng = [i+j+k+m for (i,j,k,m) in zip(*[text[i:] for i in range(4)]) if j!=' ' or k!=' ']
  #for (l,m) in zip(*[text[i:] for i in range(2)]):
   # gram=remove_punctuation(l+m)


In [ ]:
to_char('un test de text 그러나 주로 개인적인 용도로 이용되는 위키도 있는데, 이를 개인 위키라고 한다')

In [4]:
from joblib import Memory
from shutil import rmtree

# Create a temporary folder to store the transformers of the pipeline
mem = Memory(location="/tmp/cachedir", verbose=0)

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(tokenizer=to_char, ngram_range=(1,2))

x_train_counts = count_vect.fit_transform(x_train)
x_train_counts.shape

(896115, 108154)

In [ ]:
x_train_counts
#278559

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

model = Pipeline([('TfidfVectorizer', TfidfVectorizer(tokenizer=to_char, ngram_range=(1,2))),
                     ('LinearSVC', LinearSVC()),],verbose=True,memory=mem)


model.fit(x_train, y_train)  




[Pipeline] ... (step 1 of 2) Processing TfidfVectorizer, total=  15.9s


/home/amina/.local/lib/python3.9/site-packages/sklearn/pipeline.py:348: UserWarning: Persisting input arguments took 3.40s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(


[Pipeline] ......... (step 2 of 2) Processing LinearSVC, total= 1.4min


Pipeline(memory=Memory(location=/tmp/cachedir/joblib),
         steps=[('TfidfVectorizer',
                 TfidfVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function to_char at 0x7f1701ca1700>)),
                ('LinearSVC', LinearSVC())],
         verbose=True)

In [14]:
model[0].get_feature_names_out().shape

(108086,)

In [6]:
predictions = model.predict(x_test)

In [7]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 13733     11      0 ...      0      0      0]
 [     0 160373      0 ...      0      0      0]
 [     0      0   5154 ...    277      0      0]
 ...
 [     0      0     66 ...  53881      0      0]
 [     0      1      0 ...      0   6901      0]
 [     0      0      0 ...      0      0  28047]]


In [8]:
#from sklearn import metrics
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ara       0.99      0.99      0.99     13846
         ber       0.80      0.82      0.81    194880
         bul       0.92      0.64      0.76      8016
         ces       0.94      0.94      0.94     20731
         dan       0.82      0.83      0.82     17825
         deu       0.98      0.98      0.98    191456
         ell       1.00      1.00      1.00     11611
         eng       0.98      0.99      0.98    518113
         est       0.93      0.50      0.65      1226
         fas       0.99      0.99      0.99      8191
         fin       0.98      0.97      0.97     45895
         fra       0.97      0.97      0.97    164772
         gle       0.98      0.85      0.91       861
         hbs       0.91      0.82      0.86     16937
         heb       1.00      1.00      1.00     65492
         hin       1.00      1.00      1.00      4706
         hun       0.98      0.99      0.99    117260
         hye       0.99    

In [23]:
metrics.accuracy_score(y_test,predictions)

0.841213926781596

In [9]:
model.predict(['그러나 주로 개인적인 용도로 이용되는 위키도 있는데, 이를 개인 위키라고 한다. bonjour'])[0] 

'kor'

In [25]:
model.predict(['ויקי יכולה להיות שיטה טובה לשיתוף ידע בקהילות שפועלות באמצעות האינטרנט או בתוךיות. היא חלופה'])[0]

'heb'

In [10]:
model.predict(['einsetzt. Zudem nutzen auch viele '])[0]

'deu'

In [11]:
model.predict(['الكرام بتحويل الكتابة العربي الى كتابة مفهومة من قبل اغلب برامج التصميم مثل الفوتوشوب و الافترايفكتس و البريمير و الافد ميدا كومبوزر و السموك و برامج اخرى'])[0]

'ara'

In [12]:
model.predict(['से प्रत्येक एक विशिष्ट भाषा से संबंधित है। विकिपीडिया के अलावा, सार्वजनिक और निजी दोनों उपयोग में सैकड़ों हजारों अ'])[0]

'hin'

# Model persistence

In [14]:
from joblib import dump
dump(model, 'data/lang_detector.joblib') 

['data/lang_detector.joblib']